In [ ]:
# ------------------------------------------------------------------
# 1. Operations on GP/RBF surrogates
# ------------------------------------------------------------------
struct RBFsurrogate2{}
    ψ::RBFfun
    X::Matrix{Float64}
    K::Matrix{Float64}
    L::LowerTriangular{Float64, Matrix{Float64}}
    y::Vector{Float64}
    c::Vector{Float64}
    σn2::Float64
    ymean::Float64
end

function fit_surrogate2(ψ::RBFfun, X::Matrix{Float64}, y::Vector{Float64}; σn2=1e-6)
    d, N = size(X)
    K = eval_KXX(ψ, X, σn2=σn2)
    L = cholesky(Hermitian(K)).L
    ymean = mean(y)
    y .-= ymean
    c = L'\(L\y)
    return RBFsurrogate2(ψ, X, K, L, y, c, σn2, ymean)
end

# TODO: Change to a function that updates the object in place
function update_surrogate2(s::RBFsurrogate, xnew::Vector{Float64}, ynew::Float64)
    X = hcat(s.X, xnew)
    y = vcat(s.y .+ s.ymean, ynew) # Recovery y and add new observation
    ymean = mean(y) # Compute new mean of observations
    y .-= ymean # Offset observations to be zero mean

    # Update covariance matrix and it's cholesky factorization
    KxX = eval_KxX(s.ψ, xnew, s.X)
    K = [s.K  KxX
         KxX' eval_KXX(s.ψ, reshape(xnew, length(xnew), 1), σn2=s.σn2)]
    
    function update_cholesky(K::Matrix{Float64}, L::LowerTriangular{Float64, Matrix{Float64}})
        # Grab entries from update covariance matrix
        n = size(K, 1)
        B = @view K[n:n, 1:n-1]
        C = K[n, n]
        
        # Compute the updated factorizations using schur complements
        L21 = B / L'
        L22 = sqrt(C - first(L21*L21'))
    
        # Update the full factorization
        ufK = zeros(n, n)
        ufK[1:n-1, 1:n-1] .= L
        ufK[n:n, 1:n-1] .= L21
        ufK[n, n] = L22
    
        return LowerTriangular(ufK)
    end

    L = update_cholesky(K, s.L)
    c = L'\(L\y)
    return RBFsurrogate(s.ψ, X, K, L, y, c, s.σn2, ymean)
end